In [1]:
import pandas as pd 

In [2]:
def process_gov_csv(file_name): 
    df= pd.read_csv(file_name, delimiter="\t",encoding='utf-16')
    df.drop('Gender',axis=1,inplace=True)
    df.drop('Year of Birth',axis=1,inplace=True)
    df['Start']=pd.to_datetime(df['Start'],errors='coerce',format='%d-%b-%y')
    df['End']=pd.to_datetime(df['End'],errors='coerce',format='%d-%b-%y')
    #df['Start']=df['Start'].dt.strftime('-%b-%y')
    #df['End']=df['End'].dt.strftime('-%b-%y')
    return df 

In [3]:
file_name='data.csv'

In [4]:
df=process_gov_csv(file_name)

In [5]:
df.to_csv('example.csv')

 now, onto matching :D 

In [7]:
election_dates = [
    "Austria-Oct-94", "Austria-Dec-95", "Austria-Oct-99", "Austria-Nov-02",
    "Austria-Oct-06", "Austria-Sep-08", "Austria-Sep-13", "Austria-Oct-17", "Austria-Sep-19"
]
countryname = 'Austria'


In [21]:
def match_ministries_and_elections(countryname,election_dates,df): 
    sorted_elections= sorted(pd.to_datetime(election_dates, format=f'{countryname}-%b-%y'))
    election_period_dict = {}
    for i in range((len(sorted_elections))): 
        start=sorted_elections[i]
        end = sorted_elections[i+1] if i+1<len(sorted_elections) else pd.Timestamp.today()#.strftime('-%b-%y')
        boolean_mask = ((df['Start']>=start)&(df['Start']<end))
        df_i=df[boolean_mask]
        key=start.strftime('-%b-%y')
        election_period_dict[key]=df_i
    return election_period_dict
        

In [22]:
test = matching(countryname,election_dates,df)

In [23]:
test.keys()

dict_keys(['-Oct-94', '-Dec-95', '-Oct-99', '-Nov-02', '-Oct-06', '-Sep-08', '-Sep-13', '-Oct-17', '-Sep-19'])

In [31]:
df=test.get('-Sep-19')
ministries=df.loc[df['Start']==df['Start'].min(),['Position','Party']]

In [32]:
df.to_csv('example.csv')

 ['FPÖ - Freedom Party of Austria (Freiheitliche Partei Österreichs, FPÖ)', 'GA - The Greens-Green Alternative (Die Grünen-Die Grüne Alternative, GA)', 'LIF - Liberal Forum (Liberales Forum, LIF)', 'SPÖ - Social Democratic Party of Austria  (Sozialdemokratische Partei Österreichs, SPÖ)', "ÖVP - Austrian People's Party  (Österreichische Volkspartei, ÖVP)"]

In [35]:
parties =['FPÖ - Freedom Party of Austria (Freiheitliche Partei Österreichs, FPÖ)', 'GA - The Greens-Green Alternative (Die Grünen-Die Grüne Alternative, GA)', 'LIF - Liberal Forum (Liberales Forum, LIF)', 'SPÖ - Social Democratic Party of Austria  (Sozialdemokratische Partei Österreichs, SPÖ)', "ÖVP - Austrian People's Party  (Österreichische Volkspartei, ÖVP)"]

In [36]:
def get_ministry_dict(df,parties): 
    dict={}
    for party in parties: 
        dict[party]=df[df['Party']==party]['Position'].tolist()
    return dict

In [37]:
test_dict=get_ministry_dict(df,parties)

In [38]:
def get_ministry_dicts(df,parties): 
    ## takes in a df with cols position and party and a list of all parties
    ## creates 3 dicts, which store the following information
    dict_1={} #(party,list_of_ministries)
    dict_2={} #(party,number_of_ministries)
    dict_3={} #(party,weighted_number_of_ministries) ## weighting prime minister = 3, all other 1
    prime_minister=df.iloc[0]['Position'] #prime minister, chancellor whatever is always the first entry of the df, if created correctly from political yearbook data
    for party in parties: 
        ministries=df[df['Party']==party]['Position'].tolist()
        dict_1[party]=ministries
        dict_2[party]=len(ministries)
        weighted_ministries= len(ministries) + 2 if prime_minister in ministries else len(ministries)
        dict_3[party]=weighted_ministries
    return dict_1,dict_2,dict_3
        

In [40]:
ministries_by_party_dict, ministries_count_by_party, weighted_ministries_count_by_party = get_ministry_dicts(ministries, parties)